In [1]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs

from django.conf import settings

from cmj.utils import clean_text

gemini_api_key = settings.GEMINI_API_KEY

In [ ]:
import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [ ]:
def get_model_configured():
    generation_config = {
      "temperature": 0.1,
      "top_p": 0.95,
      "top_k": 40,
      #"max_output_tokens": 8192,
      "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-exp",
      #model_name="gemini-2.5-pro-exp-03-25",
      #model_name="gemini-2.5-flash-preview-04-17",
      generation_config=generation_config,
    )

    return model

model = get_model_configured()


In [ ]:
def make_prompt(num_prompt, original, analisado, o_epigrafe, a_epigrafe):

    prompt0 = f"""
Assuma a personalidade de um especialista em produção de textos legislativos em uma câmara municipal,
com experiência em redação de documentos oficiais. Sua tarefa é analisar dois textos
que são pedidos de providências feitos por vereadores e identificar se eles estão pedindo o mesmo benefício.
Os textos podem ter diferenças de redação, mas você deve se concentrar no conteúdo e na intenção dos pedidos.
Para tal tarefa, compare o conteúdo de <ORIGINAL></ORIGINAL> com o conteúdo de <ANALISADO></ANALISADO>.
Para citar estes dois conteúdos, nomeie eles respectivamente da seguinte maneira:
"{o_epigrafe}" e "{a_epigrafe}".
Remova de sua análise os autores pois são irrelevantes para a comparação requerida.
O importante é o que está sendo pedido, quem será o beneficiário do pedido e para qual localidade
está sendo feito tal pedido.
Não faça considerações adicionais e ou mesmo conclusão extra. Neste contexto responda:

- Os textos estão pedindo o mesmo benefício para a mesma localidade?
- Descreva de forma sucinta e direta o que está sendo pedido em <ORIGINAL></ORIGINAL> e <ANALISADO></ANALISADO> informando também os beneficiários e as localidades citadas.
- Calcule a semelhança percentual entre os documentos desconsiderando autores, focando na solicitação e no beneficiário, qual semelhança percentual entre <ORIGINAL></ORIGINAL> e <ANALISADO></ANALISADO>? Coloque o resultado em percentual com uma marcação fácil de ser extraída via regex "\[\[ \d{1,3}% \]\]".
- formate a resposta em MARKDOWN, utilizando linguagem dissertativa com os títulos e subtítulos necessários para facilitar a leitura, utilizando negrito e itálico quando necessário
- Não utilize a palavra "plágio" em sua resposta, se necessário expressar tal sentido, utilize a palavra "similaridade".

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompt1 = f"""    Compare o conteúdo de <ORIGINAL></ORIGINAL>, ao qual será chamado daqui em diante de "T.O.", com o conteúdo de <ANALISADO></ANALISADO>, ao qual será chamado daqui em diante de "T.A.", e escreva em formato MARKDOWN respondendo as perguntas numeradas que estão em <PERGUNTAS></PERGUNTAS>.

<PERGUNTAS>
    1) "T.O." e "T.A." fazem solicitações a seus destinatários? Quais são essas solicitações? E qual o beneficiário de cada uma das solicitações?
    2) Desconsiderando autores e destinatários, focando na solicitação e no beneficiário, qual grau de semelhança, de zero a um, poderia ser atribuído entre "T.O." e "T.A."?
</PERGUNTAS>

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompt2 = f"""    Assuma a personalidade de um especialista em plágio de documentos. É muito importante que em sua resposta nunca seja usada a palavra "plágio", em vez disso, se necessário, utilize a palavra similaridade.
    Compare o conteúdo de <ORIGINAL></ORIGINAL>, ao qual será chamado daqui em diante de "T.O.", com o conteúdo de <ANALISADO></ANALISADO>, ao qual será chamado daqui em diante de "T.A." e escreva em formato MARKDOWN respondendo as perguntas numeradas que estão em <PERGUNTAS></PERGUNTAS>. Seja Conciso.

<PERGUNTAS>
    1) Existe semelhança temática entre "T.O." e "T.A."? Quais são estas semelhanças? É o mesmo tema (frize)?
    2) Existe semelhança nas solicitações feita em "T.O." e "T.A."? Quais são estas semelhanças?
    3) A solicitações feitas é uma melhoria para algum lugar, seja esse lugar um logradouro público, um bairro, um prédio comercial, um prédio público, ou ainda instituições públicas e/ou privadas. Que lugar é este? É o mesmo lugar?
    4) Alguma consideração a fazer quanto a similaridade entre "T.O." e "T.A."?
    5) Desconsiderando autores e destinatários, focando na solicitação e no beneficiário, qual grau de semelhança, de zero a um, poderia ser atribuído entre "T.O." e "T.A."?
</PERGUNTAS>

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompts = [prompt0, prompt1, prompt2]

    prompt = prompts[num_prompt]
    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt


In [ ]:
from sapl.materia.models import MateriaLegislativa

originais = MateriaLegislativa.objects.filter(numero=370, tipo_id=3, ano=2025).order_by('id')
analisadas = MateriaLegislativa.objects.filter(numero=46, tipo_id=3, ano=2025).order_by('id')

fez_um_doc = False
for original in originais:
    for analisada in analisadas:
        if original == analisada:
            continue

        if not set(original.autores.all()) - set(analisada.autores.all()):
            continue

        fez_um_doc = True

        print(original, '≃', analisada)

        doc_original = pymupdf.open(original.texto_original.path)
        text_original = ' '.join([page.get_text() for page in doc_original])
        text_original = clean_text(text_original)

        doc_analisada = pymupdf.open(analisada.texto_original.path)
        text_analisada = ' '.join([page.get_text() for page in doc_analisada])
        text_analisada = clean_text(text_analisada)

        prompt = make_prompt(0, text_original, text_analisada, original.epigrafe_short, analisada.epigrafe_short)

        answer = model.generate_content(prompt)
        display(Markdown(answer.text))

        if fez_um_doc:
            break

    if fez_um_doc:
        break


In [6]:
from sapl.materia.models import MateriaLegislativa

requerimentos = MateriaLegislativa.objects.filter(
    tipo_id=3, ano=2025, similaridades__isnull=True
).prefetch_related('autores', 'assuntos'
                   ).order_by('-id').distinct()


In [ ]:
# cria uma lista de tuplas contendo o id do requerimento, uma tumpla com os ids dos autores e uma tupla com os ids dos assuntos
requerimentos_ids = []
for requerimento in requerimentos:
    requerimentos_ids.append(
        (requerimento.id, tuple(requerimento.autores.values_list('id', flat=True)),
         tuple(requerimento.assuntos.values_list('id', flat=True)))
    )

# ordena a lista pela quantidade de assuntos
requerimentos_ids = sorted(requerimentos_ids, key=lambda x: (len(x[2]), len(x[1])), reverse=True)

# ordena internamente cada tupla e assuntos
for i, r in enumerate(requerimentos_ids):
    requerimentos_ids[i] = (r[0], tuple(sorted(r[1])), tuple(sorted(r[2])))

# cria uma lista dois a dois de todos os requerimentos com todos menos com ele mesmo, e se o autor for o mesmo
requerimentos_comparacao = {}
for i, r1 in enumerate(requerimentos_ids):
    for j, r2 in enumerate(requerimentos_ids):
        if i == j:
            continue
        if r1[1] == r2[1]:
            continue
        if len(r1[1]) >=5 or len(r2[1]) >= 5:
            continue
        set1 = set(r1[2])
        set2 = set(r2[2])
        intersection = set1.intersection(set2)
        requerimentos_comparacao[(r1[0], r2[0])] = tuple(intersection)

# remove os requerimentos que possuem chave invertida
# ou seja, dado (a, b), remove (b, a)

requerimentos_comparados = {}
for k, v in requerimentos_comparacao.items():
    if (k[1], k[0]) in requerimentos_comparados:
        continue
    requerimentos_comparados[k] = v


# ordena a comparação pela quantidade de elementos na interseção
requerimentos_comparados = sorted(requerimentos_comparados.items(), key=lambda x: len(x[1]), reverse=True)
print(len(requerimentos_ids), len(requerimentos_comparados))

In [ ]:
for r in requerimentos_comparados[:100]:
    print(r)

In [ ]:
def get_requerimento_in_requerimentos_comparados(r):
    items = []
    for i, r1 in enumerate(requerimentos_comparados):
        if r in r1[0]:
            items.append(r1)
    return items

aparicoes = get_requerimento_in_requerimentos_comparados(21536)


In [ ]:
list(aparicoes)

In [2]:
from sapl.base.tasks import task_analise_similaridade_entre_materias_function
from sapl.materia.models import AnaliseSimilaridade

task_analise_similaridade_entre_materias_function()



In [3]:

analises = AnaliseSimilaridade.objects.filter(
    similaridade__gt=-1
    ).order_by('-similaridade')

for analise in analises:
    print(analise, analise.qtd_assuntos_comuns)
    #display(Markdown(analise.analise))
    #break

Matéria 1: Requerimento nº 62 de 2025 - Matéria 2: Requerimento nº 28 de 2025 - Similaridade: 60 6
Matéria 1: Requerimento nº 62 de 2025 - Matéria 2: Requerimento nº 131 de 2025 - Similaridade: 60 6
Matéria 1: Requerimento nº 62 de 2025 - Matéria 2: Requerimento nº 71 de 2025 - Similaridade: 0 6
Matéria 1: Requerimento nº 62 de 2025 - Matéria 2: Requerimento nº 265 de 2025 - Similaridade: 0 6
Matéria 1: Requerimento nº 62 de 2025 - Matéria 2: Requerimento nº 138 de 2025 - Similaridade: 0 6
Matéria 1: Requerimento nº 62 de 2025 - Matéria 2: Requerimento nº 215 de 2025 - Similaridade: 0 6
Matéria 1: Requerimento nº 112 de 2025 - Matéria 2: Requerimento nº 141 de 2025 - Similaridade: 0 7
Matéria 1: Requerimento nº 112 de 2025 - Matéria 2: Requerimento nº 272 de 2025 - Similaridade: 0 8
Matéria 1: Requerimento nº 62 de 2025 - Matéria 2: Requerimento nº 120 de 2025 - Similaridade: 0 6


In [3]:
from sapl.parlamentares.models import Legislatura


legislatura_atual = Legislatura.cache_legislatura_atual()

In [4]:
legislatura_atual

{'id': 21,
 'numero': 21,
 'data_inicio': datetime.date(2025, 1, 1),
 'data_fim': datetime.date(2028, 12, 31),
 'data_eleicao': datetime.date(2024, 10, 6)}

In [8]:
requerimentos = MateriaLegislativa.objects.filter(
            tipo_id=3, ano__gte=legislatura_atual['data_inicio'].year, ano__lte=legislatura_atual['data_fim'].year
        ).prefetch_related('autores', 'assuntos'
                        ).order_by('-id').distinct()

In [9]:
requerimentos.count()

277